In [3]:
from __future__ import annotations
from typing import NamedTuple, Tuple, Dict, List
from enum import Enum
from collections import Counter, defaultdict

class XY(NamedTuple):
    x: int
    y: int

class Seat(NamedTuple):
    status: str
    loc: XY
        
class Status(Enum):
    EMPTY = 0
    OCCUPIED = 1
    BLOCKED = 2
    
    @staticmethod
    def parse_status(in_str:str)->Status:
        if in_str == 'L': return Status.EMPTY
        if in_str =='#':  return Status.OCCUPIED
        if in_str == '.': return Status.BLOCKED
        return ValueError('Not a valid status')
    
    
class SeatPlan:
    
    def __init__(self, inputs:str)-> None:
        self.inputs = inputs
        self.by_loc = {seat.loc : seat.status
            for seat in self.parse_inputs()
        }
        self.rows = max(y
            for _, y in self.by_loc.keys())
        self.columns = max(x
            for x, _ in self.by_loc.keys())
        self.round = 0
            
    def parse_inputs(self)-> List[Seat]:
        return [Seat(status= Status.parse_status(in_str=col), 
              loc= XY(x=x, y=y))
    for y, row in enumerate(self.inputs.split("\n"))
    for x, col in enumerate(row)]
        
    def _get_adjacent(self) -> Dict[XY,Counter]:
        adjacent = defaultdict(list)
        for loc in self.by_loc.keys():
            x, y = loc
            if y+1 <= self.rows: adjacent[loc].append(XY(x=x, y= y+1)) #up
            if y-1 >= 0: adjacent[loc].append(XY(x=x, y=y-1)) #down
            if x+1 <= self.columns: adjacent[loc].append(XY(x=x+1, y=y))#right
            if x-1 >= 0: adjacent[loc].append(XY(x=x-1, y=y)) #left
            if y+1 <= self.rows and x+1 <= self.columns: adjacent[loc].append(XY(x=x+1, y=y+1)) #top-right
            if y+1 <= self.rows and x-1 >= 0: adjacent[loc].append(XY(x=x-1, y=y+1)) #top-left
            if y-1 >= 0 and x+1 <= self.columns: adjacent[loc].append(XY(x=x+1, y=y-1)) #bottom-right
            if y-1 >= 0 and x-1 >= 0: adjacent[loc].append(XY(x=x-1, y=y-1)) #bottom-left
        return {loc: Counter(self.by_loc[loc] for loc in seat_adj)
            for loc, seat_adj in adjacent.items()
        }
    
     
    def _get_visible(self) -> Dict[XY,Counter]:
        """
        inspired by p2 JG solution
        """
        visible = defaultdict(list)
        neighbours = [XY(-1,-1), XY(0,-1), XY(1, -1), 
                      XY(-1, 0),           XY(1, 0),
                      XY(-1, 1), XY(0, 1),  XY(1, 1)]
        for loc in self.by_loc.keys():
            for dx, dy in neighbours:
                x, y = loc
                while True:
                    x += dx
                    y += dy
                    #print("inital loc, x, y", loc, x, y)
                    if 0 <= x <= self.columns and 0<= y <= self.rows: #in seat plan
                        status = self.by_loc[XY(x,y)]
                        #print(status)
                        if status == Status.EMPTY or status == Status.OCCUPIED:
                            #print("setting visible 1 ", loc,x, y, status)
                            visible[loc].append(status) # break as soon it locates valid status
                            break
                    else:
                        #print("setting visible 2 ", loc)
                        visible[loc].append(Status.BLOCKED)
                        break
        return {loc: Counter(statuses)
            for loc, statuses in visible.items()
        }
                        
                        
    
    def apply_rules(self, part:str = 'p1')-> bool:
        if part == 'p1':
            lookup = self._get_adjacent()
            thresh = 4
        else: #p2
            lookup = self._get_visible()
            thresh = 5
        new_plan = {}
        for loc, counter in lookup.items():
            seat_status = self.by_loc[loc]
            num_occuppied = counter.get(Status.OCCUPIED, 0)
            if seat_status == Status.BLOCKED:
                new_plan[loc] = Status.BLOCKED
            elif seat_status == Status.EMPTY and not num_occuppied:
                new_plan[loc] = Status.OCCUPIED
            elif seat_status == Status.OCCUPIED and num_occuppied >= thresh:
                new_plan[loc] = Status.EMPTY
            else:
                new_plan[loc] = seat_status
        
        #print(self.round, new_plan)
        if new_plan != self.by_loc:
            self.round += 1
            self.by_loc = new_plan
            return True
        else:
            return False
    @property
    def total_occupied(self):
        return sum(val == Status.OCCUPIED for val in self.by_loc.values())
            
                
            

RAW = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

def get_occupied(RAW:str, part:str= 'p1')-> int:
    seats = SeatPlan(inputs=RAW)
    rules = seats.apply_rules(part=part)
    while rules:
        rules = seats.apply_rules(part=part)
    #print(seats.round)
    return seats.total_occupied
assert get_occupied(RAW, part='p1') == 37
assert get_occupied(RAW, part='p2') ==  26

In [4]:
with open('puzzle_inputs/day11.txt') as f:
    PUZZ = f.read()
print("part1", get_occupied(RAW=PUZZ))
print("part2", get_occupied(RAW=PUZZ, part='p2'))

part1 2424
part2 2208
